In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
from scipy import stats
from collections import deque
from scipy.optimize import minimize
from scipy.linalg import norm, inv, det
import time
import seaborn as sns
from multiprocess import Pool, cpu_count
sns.set_theme()

# Functions

In [2]:
class multivariate_exponential_hawkes(object):
    """
    Multivariate Hawkes process with exponential kernel. No events nor initial conditions considered.
    """

    def __init__(self, mu, alpha, beta, max_jumps=None, max_time=None):
        """

        Parameters
        ----------
        mu : array_like
            Baseline intensity vector. mu.shape[0] must coincide with shapes for alpha and beta.
        alpha : array_like
            Interaction factors matrix. Must be a square array with alpha.shape[0] coinciding with mu and beta.
        beta : array_like
            Decay factor matrix. Must be either an array. When corresponding to decay for each process i, it must
            be of shape (number_of_process, 1), or a square array. beta.shape[0] must coincide with mu and alpha.
        max_jumps : float, optional
            Maximal number of jumps. The default is None.
        max_time : float, optional
            Maximal time horizon. The default is None.

        Attributes
        ----------
        nb_processes : int
            Number of dimensions.
        timestamps : list of tuple (float, int)
            List of simulated events and their marks.
        intensity_jumps : array of float
            Array containing all intensities at each jump. It includes the baseline intensities mu.
        simulated : bool
            Parameter that marks if a process has been already been simulated,
            or if its event times have been initialized.

        """

        # We must begin by verifying that the process is a point process. In other words, that the number of
        # points in any bounded interval is a.s. finite. For this, we have to verify that the spectral radius of
        # the matrix alpha/beta (term by term) is <1.

        beta_radius = np.copy(beta)
        beta_radius[beta_radius == 0] = 1
        spectral_radius = np.max(np.abs(np.linalg.eig(np.abs(alpha) / beta_radius)[0]))

        if spectral_radius >= 1:
            # raise ValueError("Spectral radius is %s, which makes the process unstable." % (spectral_radius))
            warnings.warn("Spectral radius is %s, which makes the process unstable." % (spectral_radius),RuntimeWarning)
        self.mu = mu.reshape((alpha.shape[0], 1))
        self.alpha = alpha
        self.beta = beta
        self.max_jumps = max_jumps
        self.max_time = max_time

        self.nb_processes = self.mu.shape[0]
        self.count = np.zeros(self.nb_processes, dtype=int)

        self.timestamps = [(0.0, 0)]
        self.intensity_jumps = np.copy(mu)

        self.simulated = False

    def simulate(self):
        """
        Auxiliary function to check if already simulated and, if not, which simulation to launch.

        Simulation follows Ogata's adapted thinning algorithm. Upper bound obtained by the positive-part process.

        Works with both self-exciting and self-regulating processes.

        To launch simulation either self.max_jumps or self.max_time must be other than None, so the algorithm knows when to stop.
        """
        if not self.simulated:
            if self.max_jumps is not None and self.max_time is None:
                self.simulate_jumps()
            elif self.max_time is not None and self.max_jumps is None:
                self.simulate_time()
            else:
                print("Either max_jumps or max_time must be given.")
            self.simulated = True

        else:
            print("Process already simulated")

    def simulate_jumps(self):
        """
        Simulation is done until the maximal number of jumps (self.max_jumps) is attained.
        """
        flag = 0
        t = 0

        auxiliary_alpha = np.where(self.alpha > 0, self.alpha, 0)
        auxiliary_ij = np.zeros((self.nb_processes, self.nb_processes))
        auxiliary_intensity = np.copy(self.mu)

        ij_intensity = np.zeros((self.nb_processes, self.nb_processes))

        while flag < self.max_jumps:

            upper_intensity = np.sum(auxiliary_intensity)

            previous_t = t
            t += np.random.exponential(1 / upper_intensity)

            # ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - self.timestamps[-1][0])))
            ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - previous_t)))
            candidate_intensities = self.mu + np.sum(ij_intensity, axis=1, keepdims=True)
            pos_candidate = np.maximum(candidate_intensities, 0) / upper_intensity
            type_event = np.random.multinomial(1, np.concatenate((pos_candidate.squeeze(), np.array([0.0])))).argmax()
            if type_event < self.nb_processes:
                self.timestamps += [(t, type_event + 1)]
                ij_intensity[:, type_event] += self.alpha[:, type_event]
                self.intensity_jumps = np.c_[
                    self.intensity_jumps, self.mu + np.sum(ij_intensity, axis=1, keepdims=True)]

                auxiliary_ij = np.multiply(auxiliary_ij, np.exp(-self.beta * (t - self.timestamps[-2][0])))
                auxiliary_ij[:, type_event] += auxiliary_alpha[:, type_event]
                auxiliary_intensity = self.mu + np.sum(auxiliary_ij, axis=1, keepdims=True)

                flag += 1

                self.count[type_event] += 1

        self.max_time = self.timestamps[-1][0]
        # Important to add the max_time for plotting and being consistent.
        self.timestamps += [(self.max_time, 0)]

    def simulate_time(self):
        """
        Simulation is done for a window [0, T] (T = self.max_time) is attained.
        """
        t = 0
        flag = t < self.max_time

        auxiliary_alpha = np.where(self.alpha > 0, self.alpha, 0)
        auxiliary_ij = np.zeros((self.nb_processes, self.nb_processes))
        auxiliary_intensity = np.copy(self.mu)

        ij_intensity = np.zeros((self.nb_processes, self.nb_processes))

        while flag:

            upper_intensity = np.sum(auxiliary_intensity)

            previous_t = t
            t += np.random.exponential(1 / upper_intensity)

            # ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - self.timestamps[-1][0])))
            ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - previous_t)))
            candidate_intensities = self.mu + np.sum(ij_intensity, axis=1, keepdims=True)
            pos_candidate = np.maximum(candidate_intensities, 0) / upper_intensity
            type_event = np.random.multinomial(1,
                                               np.concatenate((pos_candidate.squeeze(), np.array([0.0])))).argmax()
            flag = t < self.max_time
            if type_event < self.nb_processes and flag:
                self.timestamps += [(t, type_event + 1)]
                ij_intensity[:, type_event] += self.alpha[:, type_event]
                self.intensity_jumps = np.c_[
                    self.intensity_jumps, self.mu + np.sum(ij_intensity, axis=1, keepdims=True)]

                auxiliary_ij = np.multiply(auxiliary_ij, np.exp(-self.beta * (t - self.timestamps[-2][0])))
                auxiliary_ij[:, type_event] += auxiliary_alpha[:, type_event]
                auxiliary_intensity = self.mu + np.sum(auxiliary_ij, axis=1, keepdims=True)

                self.count[type_event] += 1

        self.timestamps += [(self.max_time, 0)]

    def plot_intensity(self, ax=None, plot_N=True, where=10):
        """
        Plot intensity function. If plot_N is True, plots also step functions N^i([0,t]).
        The parameter ax allows to plot the intensity function in a previously created plot.

        Parameters
        ----------
        ax : array of Axes, optional.
            If None, method will generate own figure.
            Otherwise, will use given axes. Must be array of shape (2,K) if plot_N = True, or (K,) if plot_N = False
        plot_N : bool, optional.
            Whether we plot the step function N^i or not.
        """

        if not self.simulated:
            print("Simulate first")

        else:
            plt.rcParams['axes.grid'] = True
            if plot_N:
                jumps_plot = [[0] for i in range(self.nb_processes)]
                if ax is None:
                    fig, ax = plt.subplots(2, self.nb_processes, sharex=True)
                    ax1 = ax[0, :]
                    ax2 = ax[1, :]
                elif isinstance(ax[0,0], matplotlib.axes.Axes):
                    ax1 = ax[0, :]
                    ax2 = ax[1, :]
                else:
                    return "ax is the wrong shape. It should be (2, number of processes+1)"
            else:
                if ax is None:
                    fig, ax1 = plt.subplots(1, self.nb_processes)
                elif isinstance(ax, matplotlib.axes.Axes) or isinstance(ax, np.ndarray):
                    ax1 = ax
                else:
                    return "ax is the wrong shape. It should be (number of processes+1,)"

            times = [0, self.timestamps[1][0]]
            intensities = np.array([[self.mu[i, 0], self.mu[i, 0]] for i in range(self.nb_processes)])

            ij_intensity = np.zeros((self.nb_processes, self.nb_processes))

            step = 100
            # print("here", self.timestamps[len(self.timestamps)])

            for i in range(1, len(self.timestamps[1:where])):
                # On commence par mettre à jour la matrice lambda^{ij}
                ij_intensity = np.multiply(ij_intensity,
                                           np.exp(-self.beta * (self.timestamps[i][0] - self.timestamps[i - 1][0])))
                # On enregistre le saut d'intensité de l'évenement, pour son type.
                ij_intensity[:, self.timestamps[i][1]-1] += self.alpha[:, self.timestamps[i][1]-1]

                # On définit la fonction à tracer entre T_n et T_{n+1}
                func = lambda x: self.mu + np.matmul(
                    np.multiply(ij_intensity, np.exp(-self.beta * (x - self.timestamps[i][0]))),
                                np.ones((self.nb_processes, 1)))

                # On enregistre la division de temps et les sauts
                interval_t = np.linspace(self.timestamps[i][0], self.timestamps[i + 1][0], step)
                times += interval_t.tolist()

                intensities = np.concatenate((intensities, np.array(list(map(func, interval_t))).squeeze().T ), axis=1)
                if plot_N:
                    jumps_plot[self.timestamps[i][1]-1] += [self.timestamps[i][0] for t in range(2)]

            for i in range(self.nb_processes):
                ax1[i].plot(times, intensities[i], label="Underlying intensity", c="#1f77b4", linestyle="--")
                ax1[i].plot(times, np.maximum(intensities[i], 0), label="Conditional intensity", c='r')
                # ax1[i].plot([i for i,j in self.timestamps[:-1]], self.intensity_jumps[i,:], c='k', alpha=0.5)

            ax1[0].legend()

            if plot_N:
                for i in range(self.nb_processes):
                    jumps_plot[i] += [times[-1]]
                    ax2[i].plot(jumps_plot[i], [t for t in range(len(jumps_plot[i])//2) for j in range(2)], c="r", label="Process #%s"%(i+1))
                    # ax2[i].set_ylim(ax2[i].get_ylim())
                    for j in range(self.nb_processes):
                        if j != i:
                            ax2[j].plot(jumps_plot[i], [t for t in range(len(jumps_plot[i])//2) for j in range(2)], c="#1f77b4", alpha=0.5)

                    ax2[i].legend()

    def plot_heatmap(self, ax=None):
        """
        This function allows to observe the heatmap where each cell {ij} corresponds to the value {alpha/beta} from that interaction

        Parameters
        ----------
        ax : .axes.Axes, optional.
            If None, method will generate own ax.
            Otherwise, will use given ax.
        """
        import seaborn as sns

        if ax is None:
            fig, ax = plt.subplots()
        else:
            ax = ax
        beta_heat = np.copy(self.beta)
        beta_heat[beta_heat == 0] = 1
        heat_matrix = self.alpha/beta_heat

        hex_list = ['#FF3333', '#FFFFFF', '#33FF49']

        ax = sns.heatmap(heat_matrix, cmap=get_continuous_cmap(hex_list), center=0, ax=ax, annot=True)

In [3]:
def multivariate_spectral_density(w, theta):
    mu, alpha, beta = theta
    dim = mu.shape[0]
    mean_matrix = np.identity(dim) * (inv(np.identity(dim) - alpha + 1e-12) @ mu)
    
    fourier_matrix = alpha * beta / (beta - 2j * np.pi * w)
    spectral_matrix = inv(np.identity(dim) - fourier_matrix)
    return (spectral_matrix) @ mean_matrix @ np.conj(spectral_matrix.T)


def spectral_multivariate_ll(theta, periodogram, K, max_time):
    dim = int(np.sqrt(theta.shape[0] + 1) - 1)
    theta_aux = (theta[:dim].reshape((dim, 1)), theta[dim:-dim].reshape((dim, dim)), theta[-dim:].reshape((dim, 1)))
    f_matrixes = [multivariate_spectral_density(j/max_time, theta_aux) for j in range(1, K+1)]
    ll = np.sum([np.trace(inv(f_matrixes[i]) @ periodogram[i]) + np.log(det(f_matrixes[i])) for i in range(0, K)])
    return (1/max_time) * ll

In [4]:
def multivariate_periodogram_good(w, tList):
    max_time = tList[-1][0]
    dim = int(np.max(np.array(tList)[:, 1]))
    
    dimensional_times = [[t for t,i in tList if i == j] for j in range(1, dim+1)]
    
    J_w = np.array([np.sum([np.exp(-2j * np.pi * w * t) for t in i]) for i in dimensional_times]).reshape((dim, 1))
    return (1/max_time) * J_w @ np.conj(J_w.T)

def multivariate_periodogram_bad(w, tList):
    max_time = tList[-1][0]
    dim = int(np.max(np.array(tList)[:, 1]))
    
    dimensional_times = [[t for t,i in tList if i == j] for j in range(1, dim+1)]
    
    J_w = np.array([np.sum([np.exp(-2j * np.pi * w * t) for t in i]) for i in dimensional_times]).reshape((dim, 1))
    return (1/max_time) * np.conj(J_w) @ (J_w.T)


def multivariate_spectral_noised_density(w, theta):
    mu, alpha, beta, noise = theta
    dim = mu.shape[0]
    mean_matrix = np.identity(dim) * (inv(np.identity(dim) - alpha + 1e-12) @ mu)
    
    fourier_matrix = alpha * beta / (beta + 2j * np.pi * w)
    spectral_matrix = inv(np.identity(dim) - fourier_matrix)
    return spectral_matrix @ mean_matrix @ np.conj(spectral_matrix.T) + noise * np.identity(dim)


def spectral_multivariate_noised_ll(theta, periodogram, K, max_time):
    dim = int(np.sqrt(theta.shape[0]) - 1)
    theta_mid = theta[:-1]
    theta_aux = (theta_mid[:dim].reshape((dim, 1)), theta_mid[dim:-dim].reshape((dim, dim)), theta_mid[-dim:].reshape((dim, 1)), theta[-1])
    f_matrixes = [multivariate_spectral_noised_density(j/max_time, theta_aux) for j in range(1, K+1)]
    ll = np.sum([np.trace(inv(f_matrixes[i]) @ periodogram[i]) + np.log(det(f_matrixes[i])) for i in range(0, K)])
    return (1/max_time) * ll.real

In [5]:
def multivariate_spectral_noised_single(w, theta): #alpha_21 != 0
    mu, alpha, beta, noise = theta
    #beta = np.array([beta_aux, 0]).reshape((2, 1))
    dim = mu.shape[0]
    m_1 = mu[0,0]
    m_2 = mu[1,0] + m_1 * alpha
    

    f_11 = m_1 + noise
    f_12 = m_1 * (alpha * beta)/(beta - 2j * np.pi * w)
    f_21 = m_1 * (alpha * beta)/(beta + 2j * np.pi * w)
    f_22 = m_1 * ((alpha * beta)**2)/(beta**2 + (2 * np.pi * w)**2) + m_2 + noise
    return np.array([[f_11, f_12], [f_21, f_22]])


def spectral_multivariate_noised_ll_single(theta, periodogram, K, max_time):
    dim = 2
    #theta_mid = theta[:-1]
    theta_aux = (np.array(theta[:dim]).reshape((dim, 1)), 
                 theta[dim], 
                 theta[dim+1], 
                 theta[-1])
    f_matrixes = [multivariate_spectral_noised_single(j/max_time, theta_aux) for j in range(1, K+1)]
    ll = np.sum([np.trace(inv(f_matrixes[i]) @ periodogram[i]) + np.log(det(f_matrixes[i])) for i in range(0, K)])
    return (1/max_time) * ll.real

# Estimation

In [6]:
mu = np.array([[1.0],
               [1.0]])

alpha = np.array([[0.0, 0.0],
                  [0.5, 0.0]])

beta = np.array([[1.0],
                 [1.3]])

noise = 0.5

print("Spectral radius:", np.max(np.abs(np.linalg.eig(alpha)[0])))

Spectral radius: 0.0


In [7]:
from multiprocess import Pool, cpu_count
# Reduced model estimation (parallel) Good

max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, None), 
          (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.uniform(0, 3, 5)
    
    periodogram = [multivariate_periodogram_good(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll_single,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations_max = np.array(p.map(job, range(repetitions)))
print(estimations_max)
print(estimations_max.mean(axis=0))

|-----|
|-----[[8.04391699e-01 7.07924065e-01 7.70970933e-01 1.16643960e+00
  6.60687520e-01]
 [1.46170525e+00 1.72482212e+00 2.35983817e-01 1.97789173e+00
  1.00000000e-16]
 [3.91540886e-01 5.27518394e-01 1.09868181e+00 1.22473443e+00
  1.05261324e+00]
 [8.68772653e-01 1.00339625e+00 4.77232705e-01 2.23202514e+00
  5.98577835e-01]
 [1.50297040e+00 1.49331815e+00 3.20984519e-01 1.58413529e+00
  1.00000000e-16]]
[1.00587618 1.0913958  0.58077076 1.63704524 0.46237572]


In [8]:
from multiprocess import Pool, cpu_count
# Reduced model estimation (parallel) Bad

max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, None), 
          (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.uniform(0, 3, 5)
    
    periodogram = [multivariate_periodogram_bad(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll_single,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations_max = np.array(p.map(job, range(repetitions)))
print(estimations_max)
print(estimations_max.mean(axis=0))

|-----|
|-----[[6.82092420e-03 4.78688728e-01 6.99446438e+00 3.05195551e+00
  1.45825623e+00]
 [3.74861078e-01 9.84338385e-01 1.00000000e-16 2.40785193e+00
  1.08684414e+00]
 [1.44415586e+00 2.71287731e-01 1.20778936e+00 2.14860117e-03
  1.00000000e-16]
 [2.91880274e-01 8.44934027e-01 1.00000000e-16 1.41798100e+00
  1.17547116e+00]
 [3.63419856e-04 4.61895120e-01 3.34201871e+01 3.33189912e-01
  1.50260425e+00]]
[0.42361631 0.6082288  8.32448816 1.44262539 1.04463516]


In [32]:
# Full model estimation (parallel) good
max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.rand(9)/2 + np.r_[.75, .75, 0., 0., 0., 0., .75, .75, .75]
    
    periodogram = [multivariate_periodogram_good(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations = np.array(p.map(job, range(repetitions)))
print('|\n Done')
print(estimations)
print(estimations.mean(axis=0))

|-----|
|-----|
 Done
[[1.41159461e+00 1.29973177e+00 1.00000000e-16 2.68702085e-02
  4.02243830e-01 5.01557528e-02 2.95385583e+00 1.22506070e+00
  1.00000000e-16]
 [1.40050260e+00 1.72473962e+00 1.00000000e-16 2.95677902e-02
  2.36041570e-01 1.00000000e-16 3.61299155e-01 1.97690699e+00
  1.00000000e-16]
 [1.33950023e+00 1.46226425e+00 6.75674759e-02 1.00000000e-16
  2.75144591e-01 7.60037305e-02 4.30800960e+00 1.32612993e+00
  1.00000000e-16]
 [8.69599370e-01 1.00414837e+00 1.00000000e-16 1.00000000e-16
  4.76795689e-01 4.22529772e-05 8.45508432e-01 2.23188091e+00
  5.97752887e-01]
 [1.46935398e+00 1.48755953e+00 3.28564845e-03 1.08483357e-02
  3.22241590e-01 1.00000000e-16 1.32052422e+01 1.57679922e+00
  6.23558479e-03]]
[1.29811016 1.39568871 0.01417062 0.01345727 0.34249345 0.02524035
 4.33478304 1.66735555 0.12079769]


In [33]:
# Full model estimation (parallel) bad
max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.rand(9)/2 + np.r_[.75, .75, 0., 0., 0., 0., .75, .75, .75]
    
    periodogram = [multivariate_periodogram_bad(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations = np.array(p.map(job, range(repetitions)))
print('|\n Done')
print(estimations)
print(estimations.mean(axis=0))

|-----|
|-----|
 Done
[[9.45959446e-01 1.79370733e+00 1.00000000e-16 2.60740057e-01
  2.89011036e-02 7.49201763e-02 1.22102990e+00 3.26424047e+00
  1.00000000e-16]
 [4.41349857e-01 1.40067929e+00 1.00000000e-16 2.48707815e-01
  1.00000000e-16 1.00000000e-16 1.98396551e+00 5.35961373e-01
  6.70504703e-01]
 [7.69808394e-01 1.43343025e+00 7.87493824e-02 1.54809560e-01
  1.00000000e-16 1.53710342e-01 3.11169456e+00 8.27501565e-01
  3.17606196e-01]
 [1.09144610e+00 1.87479322e+00 1.00000000e-16 1.85547704e-01
  1.00000000e-16 7.06698514e-02 2.54439117e+00 1.18621304e+00
  1.00000000e-16]
 [1.04514436e+00 1.79623517e+00 1.00000000e-16 2.30808676e-01
  1.00000000e-16 9.12814046e-02 1.68943754e+00 3.72968375e-01
  1.00000000e-16]]
[0.85874163 1.65976905 0.01574988 0.21612276 0.00578022 0.07811635
 2.11010374 1.23737696 0.19762218]


In [12]:
# Without noise model estimation (parallel)
max_time = 3000
repetitions = 50
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), 
          (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    
    #init_a = np.random.uniform(0,3, dim*2)
    #a = np.random.uniform(0,3, (dim,dim))
    #radius = np.max(np.abs(np.linalg.eig(a)[0]))
    #div = np.random.uniform(1e-16, 1 - 1e-16)
    #init_alpha = a * div / (radius)
    
    #init = np.concatenate((init_a[0:dim].ravel(), init_alpha.ravel(), init_a[dim:].ravel()))
    init = np.random.rand(8)/2 + np.r_[.75, .75, 0., 0., 0., 0., .75, .75]
    
    periodogram = [multivariate_periodogram(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(4) as p:
    estimations_nonoised_model = np.array(p.map(job, range(repetitions)))
print('|\n Done')
#print(estimations)

|--------------------------------------------------|
|

/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv

--------------------------------------------------|
 Done


In [13]:
import pickle
explanation = "Estimations for Scenario (Single)."
explanation += " First is explanation, second is estimations in non-noised model,"
explanation += " Dimensions (repetitions, parameters)."
explanation
toSave = (explanation, estimations_nonoised_model)

In [14]:
with open('estimationsSingleNonNoisedModel2410.pkl', 'wb') as file:
    pickle.dump(toSave, file)